### Calculation of pairwise language distances
***
***

Set working directory (adapt accordingly) and load tidyverse for better dataframes.

In [1]:
setwd("C:/Users/stein/Dropbox/Studium/7. Semester/BA-Thesis/BA-Thesis_NorthEuraLex")

library(tidyverse)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


***
If necessary, download the necessary TSV-files from the NorthEuraLex website. Then read in the data.

In [2]:
conceptdata_loc = "./data/northeuralex-0.9-forms.tsv"
geodata_loc = "./data/northeuralex-0.9-language-data.tsv"

if(!file.exists(conceptdata_loc)) {
    download.file(
        "http://www.sfs.uni-tuebingen.de/~jdellert/northeuralex/0.9/northeuralex-0.9-forms.tsv",
        dest = conceptdata_loc
    )
}

if(!file.exists(geodata_loc)) {
    download.file(
        "http://www.sfs.uni-tuebingen.de/~jdellert/northeuralex/0.9/northeuralex-0.9-language-data.tsv",
        dest = geodata_loc
    )
}

raw_conceptdata = read_tsv("./data/northeuralex-0.9-forms.tsv", show_col_types = FALSE)
raw_geodata = read_tsv("./data/northeuralex-0.9-language-data.tsv", show_col_types = FALSE)

head(raw_conceptdata)
sample_n(raw_geodata, 3)

Language_ID,Glottocode,Concept_ID,Word_Form,rawIPA,IPA,ASJP,List,Dolgo,Next_Step
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
fin,finn1318,Auge::N,silmä,silmæ,s i l m æ,silmE,SILME,SVRMV,validate
fin,finn1318,Ohr::N,korva,kɔrʋɑ,k ɔ r ʋ ɑ,korwa,KURWA,KVRWV,validate
fin,finn1318,Nase::N,nenä,nɛnæ,n ɛ n æ,nEnE,NENE,NVNV,validate
fin,finn1318,Mund::N,suu,suː,s u u,su,SY,SV,validate
fin,finn1318,Zahn::N,hammas,hɑmːɑs,h ɑ m m ɑ s,hamas,HAMAS,HVMVS,validate
fin,finn1318,Zunge::N,kieli,kiɛ̯li,k i ɛ l i,kiEli,KIELI,KVRV,validate


name,glotto_code,iso_code,family,subfamily,latitude,longitude
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Hindi,hind1269,hin,Indo-European,Indo-Aryan,27.2000,78.000
French,stan1290,fra,Indo-European,Italic,48.0000,2.000
Central Siberian Yupik,cent2128,ess,Eskimo-Aleut,Eskimo,63.4308,-170.272


***
Filter the conceptdata for rows which have "validate" in their _Next_Step_ column, thereby excluding rows that still need to be reviewed.

In [3]:
conceptdata = raw_conceptdata %>% filter(Next_Step == "validate")

In [4]:
conceptdata = conceptdata %>% select(Language_ID, Concept_ID, Word_Form, rawIPA, ASJP, Next_Step)
head(conceptdata)

Language_ID,Concept_ID,Word_Form,rawIPA,ASJP,Next_Step
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
fin,Auge::N,silmä,silmæ,silmE,validate
fin,Ohr::N,korva,kɔrʋɑ,korwa,validate
fin,Nase::N,nenä,nɛnæ,nEnE,validate
fin,Mund::N,suu,suː,su,validate
fin,Zahn::N,hammas,hɑmːɑs,hamas,validate
fin,Zunge::N,kieli,kiɛ̯li,kiEli,validate


***
Only select the three columns needed for the calculation of the pairwise distances.

In [5]:
conceptdata = conceptdata %>% select(Language_ID, Concept_ID, ASJP)
head(conceptdata)

Language_ID,Concept_ID,ASJP
<chr>,<chr>,<chr>
fin,Auge::N,silmE
fin,Ohr::N,korwa
fin,Nase::N,nEnE
fin,Mund::N,su
fin,Zahn::N,hamas
fin,Zunge::N,kiEli


***
Make a vector with the ASJP word list and filter the data to only keep rows with one of the 40 ASJP concepts. 

In [6]:
asjp_concepts = c("Auge::N", "Ohr::N", "Nase::N", "Zahn::N", "Zunge::N",
                  "Busen::N", "Hand::N", "Knie::N", "Haut::N", "Blut::N", 
                  "Knochen::N", "Leber::N", "Sonne::N", "Stern::N", "Wasser::N",
                  "Stein::N", "Feuer::N", "Berg::N", "Baum::N", "Blatt::N",
                  "Horn::N", "Hund::N", "Fisch::N", "Laus::N", "Mensch::N", 
                  "Name::N", "Pfad::N", "Nacht::N", "voll::A", "neu::A", 
                  "ich::PRN", "du::PRN", "wir::PRN", "eins::NUM", "zwei::NUM",
                  "trinken::V", "sterben::V", "kommen::V", "sehen::V", 
                  "hören::V")

conceptdata = filter(conceptdata, Concept_ID %in% asjp_concepts)
head(conceptdata)

Language_ID,Concept_ID,ASJP
<chr>,<chr>,<chr>
fin,Auge::N,silmE
fin,Ohr::N,korwa
fin,Nase::N,nEnE
fin,Zahn::N,hamas
fin,Zunge::N,kiEli
fin,Busen::N,rinta


***
Get the iso codes of all the languages.

In [7]:
languages = raw_geodata %>% select(iso_code)
head(languages)

iso_code
<chr>
fin
krl
olo
vep
ekk
liv


***
For each language, compile a list of the concepts and the corresponding word(s) for that concept.

In [8]:
language_params = list()
for (i in 1:nrow(languages)) {
    lang = 
        conceptdata %>%
        filter(Language_ID == languages$iso_code[i]) %>%
        select(Concept_ID, ASJP)
    language_params[[i]] = lang
}

***
Create a matrix to store the pairwise language distances.

In [9]:
ldn_dists = 
    matrix(0, nrow(languages), nrow(languages))

***
Vectorize the *adist* function to use it with the *mutate* function from the *dplyr* package. *Adist* calculates the Levenshtein distance of two words.

In [10]:
adist_v = Vectorize(adist)

***
This next block calculates the normalized Levenshtein distance (LDN) between words for the same concept & then averages across all 40 concepts, to return pairwise language distances (Like in the prior paper, but with more concise code, taking advantage of tidyverse's tibbles; should take about two minutes to run).

In [11]:
for (i in 1:(nrow(languages) - 1)) {
    for (j in (i + 1):nrow(languages)) {
        # creates a tibble comparing two languages
        joined_tibble = 
            inner_join(language_params[[i]], language_params[[j]], by=c("Concept_ID" = "Concept_ID")) %>%
      
            # calculate the LND between words representing the same concept (vectorized adist comes into play here)
            mutate(avg_levenshtein_dist = adist_v(ASJP.x, ASJP.y) / pmax(nchar(ASJP.x), nchar(ASJP.y))) %>%
      
            # if there are multiple word pairs for a concept, only keep the pair with the lowest distance
            group_by(Concept_ID) %>%
            slice_min(order_by = avg_levenshtein_dist, with_ties = F)
    
    # the mean of all the concept distances is the language distance
    ldn_dists[i, j] = ldn_dists[j, i] = mean(joined_tibble$avg_levenshtein_dist)
    }
}

head(ldn_dists)

0.0000000,0.1331548,0.2214286,0.3420238,0.3494643,0.4164286,0.6958929,0.7836310,0.7352381,0.7301190,⋯,0.9576058,0.9422619,0.8884005,0.8855782,0.8780566,0.8749660,0.9318182,0.9045238,0.9308333,0.8645238
0.1331548,0.0000000,0.1704167,0.3068750,0.2984524,0.4074405,0.6870238,0.7527976,0.7094048,0.7019048,⋯,0.9464947,0.9525132,0.8923077,0.9097279,0.8819820,0.8885714,0.9385749,0.8962500,0.9258333,0.8675000
0.2214286,0.1704167,0.0000000,0.2579167,0.3358333,0.4270833,0.7066071,0.7757143,0.7291071,0.7170833,⋯,0.9685185,0.9434524,0.8970085,0.8990476,0.8747748,0.8857143,0.9335790,0.8958333,0.9166667,0.8562500
0.3420238,0.3068750,0.2579167,0.0000000,0.2787202,0.3767560,0.7005655,0.8084226,0.7714286,0.7441369,⋯,0.9564815,0.9383598,0.8833333,0.9144898,0.9058559,0.9245238,0.9416871,0.8858333,0.9179167,0.8914583
0.3494643,0.2984524,0.3358333,0.2787202,0.0000000,0.4129167,0.7494048,0.7984226,0.7551190,0.7358929,⋯,0.9439815,0.9250992,0.8749084,0.8936735,0.8843629,0.9111565,0.9369662,0.8901190,0.9142857,0.8660119
0.4164286,0.4074405,0.4270833,0.3767560,0.4129167,0.0000000,0.7159821,0.8024107,0.7757738,0.7560714,⋯,0.9337963,0.9455357,0.9110501,0.8944218,0.9073359,0.9102041,0.9376740,0.8978571,0.9172024,0.8950000


***
Save the distances in a csv file.

In [12]:
write.csv(ldn_dists, "./language_distances/ldn_dists.csv", row.names = FALSE)

***
Create a Matrix for the Levenshtein distance normalized divided (LDND).

In [13]:
ldnd_dists =
    matrix(0, nrow(languages), nrow(languages))

***
Starts with the same code block as the calculation of LDN. But additionally, all non-synonymous word pair distances are also computed. The LDN is then divided by the mean of these non-synonymous distances, resulting in LDND (may take 5-10 minutes).

In [14]:
for (i in 1:(nrow(languages) - 1)) {
    for (j in (i + 1):nrow(languages)) {
        joined_tibble = 
            inner_join(language_params[[i]], language_params[[j]], by=c("Concept_ID" = "Concept_ID")) %>%
            mutate(avg_levenshtein_dist = adist_v(ASJP.x, ASJP.y) / pmax(nchar(ASJP.x), nchar(ASJP.y))) %>%
            group_by(Concept_ID) %>%
            slice_min(order_by = avg_levenshtein_dist, with_ties = F)
      
    gamma = 0
    for (k in 1:nrow(joined_tibble)) {
          for (l in 1:nrow(joined_tibble)) {
              if(k == l){next}
              # gamma is tracking the summed-up LDNs of non-synonymous word pairs
              gamma = 
                  gamma + 
                  adist(joined_tibble$ASJP.x[k], joined_tibble$ASJP.y[l]) / 
                  max(nchar(joined_tibble$ASJP.x[k]), nchar(joined_tibble$ASJP.y[l]))
        }
    }
    # divide by the number of calculated LDNs to get the average LDN for non-synoymous word pairs
    gamma = gamma / (nrow(joined_tibble) * (nrow(joined_tibble) - 1))
    # divide the LDNs by gamma, to get LDND
    ldnd_dists[i, j] = ldnd_dists[j, i] =
        mean(joined_tibble$avg_levenshtein_dist) / gamma
  }
}

head(ldnd_dists)

0.0000000,0.1579734,0.2620130,0.3931308,0.3968384,0.4709688,0.8080191,0.8782206,0.8490529,0.8404573,⋯,1.013777,1.015846,0.9900241,0.9775972,0.9381607,0.9940535,0.9870627,0.9941389,1.0154748,0.9449136
0.1579734,0.0000000,0.2036281,0.3550604,0.3404945,0.4620941,0.7994501,0.8480265,0.8270263,0.8120653,⋯,1.008669,1.025167,0.9959903,1.0064391,0.9457668,1.0127221,0.9926387,0.9943814,1.0167547,0.9473905
0.2620130,0.2036281,0.0000000,0.2974513,0.3813807,0.4811718,0.8229075,0.8636491,0.8464200,0.8291312,⋯,1.019086,1.013200,0.9999212,0.9907112,0.9352526,1.0091758,0.9879100,0.9802088,1.0026879,0.9405365
0.3931308,0.3550604,0.2974513,0.0000000,0.3117212,0.4254044,0.7987945,0.9032555,0.8704120,0.8407602,⋯,1.023395,1.006924,0.9651291,1.0012267,0.9716997,1.0229981,0.9929986,0.9729577,1.0106634,0.9608312
0.3968384,0.3404945,0.3813807,0.3117212,0.0000000,0.4600719,0.8242966,0.8804042,0.8534376,0.8359287,⋯,1.010537,1.007382,0.9671672,0.9794530,0.9630825,1.0266077,0.9936553,0.9926387,0.9938399,0.9289986
0.4709688,0.4620941,0.4811718,0.4254044,0.4600719,0.0000000,0.7963237,0.8984228,0.8719805,0.8554732,⋯,1.007992,1.016376,0.9945735,0.9784430,0.9774791,1.0079386,0.9986236,0.9926698,1.0100068,0.9641235


***
Create another csv file.

In [15]:
write.csv(ldnd_dists, "./language_distances/ldnd_dists.csv", row.names = FALSE)

***
To calculate the next scores, the PMI scores between the ASJP sound classes are needed. If necessary, download the file with the PMI scores, then read them in.

In [11]:
PMI_data_loc = "./data/pnas.1500331112.sd04.csv"


if(!file.exists(PMI_data_loc)) {
  download.file(
    "http://www.pnas.org/lookup/suppl/doi:10.1073/pnas.1500331112/-/DCSupplemental/pnas.1500331112.sd04.csv",
    dest = PMI_data_loc
  )
}


PMI_scores = read.table("./data/pnas.1500331112.sd04.csv", sep = ",", check.names=FALSE)

head(PMI_scores)

,!,3,4,5,7,8,C,E,G,L,⋯,q,r,s,t,u,v,w,x,y,z
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
!,5.1221920,-5.366628,-0.6660987,-3.8046271,-4.727376,-2.5387277,-4.173302,-4.8432118,-0.2110367,-3.087661,⋯,-3.633199,-5.777114,-5.4221885,-6.020425375,-6.4771753,-3.733401,-5.1084969,-4.0021361,-5.133982,-3.478218
3,-5.3666279,2.321213,-4.0673450,-4.9032883,-7.435475,-5.2468267,-6.881401,-0.5782361,-3.6122829,-6.488907,⋯,-5.088535,-8.485213,-7.0316753,-8.728524451,-0.6736969,-6.441500,-8.5097432,-7.4033824,-5.167933,-5.780852
4,-0.6660987,-4.067345,5.2679133,0.8958533,-3.428093,-0.1408324,-2.874019,-3.5439288,1.0882463,-1.788378,⋯,-2.333916,-4.477831,-4.1229056,-4.721142391,-5.1778923,-2.434118,-3.1160668,-2.7028531,-3.834699,-2.178935
5,-3.8046271,-4.903288,0.8958533,3.9886589,-4.264036,-4.3779731,-3.373490,-6.6824572,-1.3571349,-2.624321,⋯,-5.472445,-4.182373,-7.2614340,-6.761058514,-8.3164207,-5.572647,-6.2545952,-5.8413815,-1.054334,-2.544875
7,-4.7273756,-7.435475,-3.4280926,-4.2640360,3.702041,-4.2021093,-4.855854,-5.6592956,-2.9730306,-2.111985,⋯,2.268349,-2.326502,-1.3543887,-0.653834114,-9.2391692,-5.396783,-0.1478133,-0.3572501,-1.950556,-4.448453
8,-2.5387277,-5.246827,-0.1408324,-4.3779731,-4.202109,4.2922055,-2.348752,-4.7234106,-0.7843826,1.221795,⋯,-3.513398,-0.785940,0.6456476,-0.002097624,-7.0505213,-3.208135,-5.6818429,-3.8823349,1.012892,1.701008


***
Next, a function to calculate the affine gap score between two words is needed.

The following R code has been adapted from the _affine_ function in the python **py_stringmatching** package; https://github.com/anhaidgroup/py_stringmatching/blob/master/py_stringmatching/similarity_measure/cython/cython_affine.pyx). 

The _gap_start_ and _gap_continuation_ values are taken from the paper _Phylogenetic Inference from Word Lists Using Weighted Alignment with Empirically Determined Weights_ by Gerhard Jäger (https://brill.com/view/journals/ldc/3/2/article-p245_4.xml?language=en)

In [12]:
affine <- function(word1, word2, gap_start = -2.4930, gap_continuation = -1.7057) {
  
  length1 = nchar(word1)
  length2 = nchar(word2)
  
  m = matrix(0, length1 + 1, length2 + 1)
  x = matrix(0, length1 + 1, length2 + 1)
  y = matrix(0, length1 + 1, length2 + 1)
  
  for (i in 2:(length1 + 1)){
    m[i, 1] = -Inf
    x[i, 1] = gap_start + (i - 1) * gap_continuation
    y[i, 1] = -Inf
  } 
  
  for (j in 2:(length2 + 1)){
    m[1, j] = -Inf
    x[1, j] = -Inf
    y[1, j] = gap_start + (j - 1) * gap_continuation
  }
  
  for (i in 2:(length1 + 1)){
    for (j in 2:(length2 + 1)){
      m[i,j] = PMI_scores[substr(word1, i - 1, i - 1), substr(word2, j - 1, j - 1)] +
        max(m[i - 1, j - 1], x[i - 1, j - 1], y[i - 1, j - 1])
      
      x[i, j] = max(gap_start + m[i - 1, j], gap_continuation + x[i - 1, j])
      
      y[i, j] = max(gap_start + m[i, j - 1], gap_continuation + y[i, j - 1])      
    }
  }
  
  max(m[length1 + 1, length2 + 1], x[length1 + 1, length2 + 1], y[length1 + 1, length2 + 1])
}

***
The following two examples are taken from the paper _Support for linguistic macrofamilies from weighted sequence alignment_, also by Gerhard Jäger (https://www.pnas.org/doi/epdf/10.1073/pnas.1500331112). The affine gap results are identical to the results in the paper, therefore the implementation of the algorithm seems to be correct.

In [13]:
affine("hEnd", "hant")
affine("mano", "hant")

[1] 4.804356

[1] -11.28492

***
Before the distances between the languages are calculated, a matrix to store them is created.

In [14]:
pmi_based_dists = matrix(1, nrow(languages), nrow(languages))

***
Vectorize the affine function to use it with mutate.

In [15]:
affine_v = Vectorize(affine)

***
Now it's time to iterate through all language pairs and store the resulting language distances in the newly created matrix. The first block is again identical to the LDN & LDND calculations (Beware, the runtime of this block is very long. )

In [16]:
for (i in 1:(nrow(languages) - 1)) {
    for (j in (i + 1):nrow(languages)) {
        joined_tibble = 
          inner_join(language_params[[i]], language_params[[j]], by=c("Concept_ID" = "Concept_ID")) %>%
          mutate(similarity = affine_v(ASJP.x, ASJP.y)) %>%
          group_by(Concept_ID) %>%
          slice_min(order_by = similarity, with_ties = F)
    
    # compute all non-synonymous word pair distances
    m = matrix(0, nrow(joined_tibble), nrow(joined_tibble))
    diag(m) = NaN
    for (k in 1:nrow(joined_tibble)) {
        for (l in 1:nrow(joined_tibble)) {
            if(k == l){next}
            m[k, l] = affine(joined_tibble$ASJP.x[k], joined_tibble$ASJP.y[l])
        }
    }
    
    # number of non_synonymous wordpairs
    non_synonymous_wordpairs = nrow(joined_tibble) * (nrow(joined_tibble) - 1)
    # calculate the calibrated similarity for each word pair
    for (k in 1:nrow(joined_tibble)){
        numerator = 1 + sum(m > joined_tibble$similarity[k], na.rm = T)
        denominator = 1 + non_synonymous_wordpairs
        calibrated_sim = -log(numerator / denominator)
        joined_tibble$similarity[k] = calibrated_sim
    }
    # calculate the pairwise distance of all languages, based on the mean concept distance  
    pmi_based_dists[i, j] = pmi_based_dists[j, i] = 
        log(log(non_synonymous_wordpairs)) - log(mean(joined_tibble$similarity))
  }
}

head(pmi_based_dists)

1.0000000,0.2020306,0.2671554,0.3877505,0.2987612,0.4980926,0.9501134,1.130384,1.182634,1.143173,⋯,2.051201,2.049683,1.955936,1.729659,1.717902,2.156545,1.942527,2.093353,2.143394,1.638936
0.2020306,1.0000000,0.2312442,0.3928244,0.2495400,0.4996734,0.8892533,1.056352,1.139539,1.097643,⋯,2.128409,2.051136,2.065297,1.855659,1.829116,2.220592,1.893201,2.079829,2.106067,1.743064
0.2671554,0.2312442,1.0000000,0.3705025,0.3002269,0.4507320,0.9490869,1.126955,1.171226,1.141359,⋯,2.129995,1.962520,2.012011,1.764128,1.752981,2.128719,1.961556,1.896714,2.076663,1.701123
0.3877505,0.3928244,0.3705025,1.0000000,0.3734886,0.5237560,1.0676698,1.252276,1.231458,1.197169,⋯,2.049211,2.127150,2.073315,1.951704,1.923538,2.306704,2.020159,2.193510,2.124377,1.882367
0.2987612,0.2495400,0.3002269,0.3734886,1.0000000,0.4186482,0.9189513,1.030004,1.110358,1.121856,⋯,1.979511,1.983815,2.054011,1.751842,1.797844,2.251170,1.913400,2.025316,2.025773,1.746150
0.4980926,0.4996734,0.4507320,0.5237560,0.4186482,1.0000000,1.0113933,1.120942,1.170445,1.121669,⋯,2.143556,1.995187,2.098740,1.813570,1.867573,2.212761,1.883280,2.218564,2.241535,1.869677


***
And save these distances aswell.

In [17]:
write.csv(pmi_based_dists, "./language_distances/pmi_dists.csv, row.names = FALSE)